# Welcome to my LSTM Model! Hit run and watch the step-by-step training of my stock prediction Model

At the end you will get a LSTM_model file which can be plugged into a different script @/stockpredictionapp.ipynb which can be applied to all stocks and not just the 9 tested here!

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

# Define the tickers to fetch data for
tickers = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'NVDA', 'ADBE', 'INTC', 'CSCO', 'IBM']

# Fetch data for each ticker using Yahoo Finance API
data = {}
for ticker in tickers:
    stock = yf.Ticker(ticker)
    data[ticker] = stock.history(period='5y')['Close']

# Combine data for all stocks into a single dataset
df = pd.concat(data.values(), axis=1, keys=data.keys())

df

# This data will now be shaped so it can be inputted into the machine learning model!

Using MinMaxScaler we are able to normalize our dataframe into managble variables which can be inputted into the model

# Creating Datasets

What is lookback?

    It is the value of N past time steps which the dataset will input into the neural network to predict the next value of the time-series.
    
This code will take that value and create datasets for testing and training the prediction values and the model.
    
    

In [ ]:
# create input/output arrays for training and testing data
def create_dataset(data, lookback):
    X, Y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i-lookback:i, 0])
        Y.append(data[i, 0])
    return np.array(X), np.array(Y)

lookback = 60
x_train, y_train = create_dataset(train_data, lookback)
x_test, y_test = create_dataset(test_data, lookback)


# Time to build the model!
1. creating the models
2. compliing the model
3. fitting the data into the model

In [ ]:

# Build Model
model = Sequential() 
model.add(LSTM(50, return_sequences=True, input_shape=(lookback, 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)


# Heres what our model looks like so far!

In [ ]:
print(model.summary())

# Now that the model is created, we can start to make our predictions!


In [ ]:
# make predictions and evaluate performance
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
mae = mean_absolute_error(y_test, predictions)
print('Mean Absolute Error:', mae)

# pickle data
model.save("lstm_model_l1")
